In [ ]:
# pip install --user --upgrade git+https://github.com/himanshudabas/twint.git@origin/twint-fixes#egg=twint

In [ ]:
import numpy as np
import pandas as pd
import twint
import time
# from datetime import datetime
from tqdm import tqdm

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 199)
pd.options.display.float_format = '{:.2f}'.format

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

import nest_asyncio
nest_asyncio.apply()

# PATHS

In [ ]:
split = 0

In [ ]:
THIS_SPLIT_PATH = fr"C:/Users/mikha/Dropbox/mikhael_misc/Projects/594/Twitter-Conspiracies/Geo Cross Referencing/User Account Info Lookup/Mikhael Splits for Searching/Split {split}.csv"

# SAVE_SPLIT_PATH = fr"C:/Users/mikha/Dropbox/mikhael_misc/Projects/594/Twitter-Conspiracies/Geo Cross Referencing/User Account Info Lookup/Searched User Results/Split {split}.csv"

# Import Split List

In [ ]:
local_split_users = pd.read_csv(THIS_SPLIT_PATH)

In [ ]:
cols_to_return = ['location', 'following', 'followers', 'likes', 'tweets', 'private', 'verified', 'join_date']

In [ ]:
cols_to_overwrite = cols_to_return + ['Searched - Bio', 'Error - Bio']

In [ ]:
#create new columns if they aren't there yet
if not all(x in local_split_users.columns.tolist() for x in cols_to_overwrite):
    local_split_users[cols_to_overwrite] = np.nan

# Define Lookup Function

In [ ]:
def get_user_info(ID):
    try:
        c = twint.Config()
        c.User_id = ID
        c.Hide_output = True
        c.Pandas = True

        twint.run.Lookup(c)
        return twint.storage.panda.User_df[cols_to_return].values.tolist()[0]
    except:
        return [np.nan]*len(cols_to_return)

# Apply Lookup Function

In [ ]:
# DELETE THIS
# local_split_users = MASTER_USER_LIST.head()

In [ ]:
local_split_users = local_split_users.set_index('ID')

In [ ]:
# user_lookup_dict = {}

for i, ID in enumerate(local_split_users.index.astype(str).tolist()):
    if local_split_users.loc[ID, 'Searched - Bio'] == 0:
        print(i, ID)
        if i % 100 == 0:
            try:
                local_split_users.loc[int(ID), cols_to_overwrite] = get_user_info(ID) + [1, 0]
    #             user_lookup_dict[ID] = get_user_info(ID)
            except:
                local_split_users.loc[int(ID), ['Searched - Bio', 'Error - Bio']] = [1, 1]
    #             user_lookup_dict[ID] = [np.nan]*len(cols_to_return)
            local_split_users.to_csv(THIS_SPLIT_PATH)


        else:
            try:
                local_split_users.loc[int(ID), cols_to_overwrite] = get_user_info(ID) + [1, 0]        
            except:
                local_split_users.loc[int(ID), ['Searched - Bio', 'Error - Bio']] = [1, 1]